# TimeMixer++ Greenhouse Experiment

This notebook trains and evaluates the TimeMixer++ baseline on the greenhouse dataset.

In [ ]:
from pathlib import Path
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt

if str(Path.cwd()) not in sys.path:
    sys.path.append(str(Path.cwd()))
    
scheme_root = Path.cwd().resolve().parent.parent.parent # if cwd is TimeMixer++
if not (scheme_root / 'TPLC_Net').exists():
    scheme_root = Path('../../../').resolve()
    
tplc_path = scheme_root / 'TPLC_Net'
if str(tplc_path) not in sys.path:
    sys.path.insert(0, str(tplc_path))

from tplc_algo.pipeline import prepare_greenhouse_datasets, make_loaders
from tplc_algo.train import Trainer, TrainConfig
from tplc_algo.utils import seed_everything
from tplc_algo.exp_utils import create_run_dir, save_metrics_json

try:
    from timemixer_pp import TimeMixerPPForecaster, TimeMixerPPConfig
except ImportError:
    sys.path.append(str(Path.cwd()))
    from timemixer_pp import TimeMixerPPForecaster, TimeMixerPPConfig

seed_everything(42)
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
dataset_root = scheme_root / 'datasets' / '自主温室挑战赛'
team = 'AICU'
seq_len = 288
pred_len = 72
stride = 1
batch_size = 32

cfg = TimeMixerPPConfig(
    seq_len=seq_len,
    pred_len=pred_len,
    enc_in=8,  # placeholder
    c_out=8,
    d_model=64,
    d_ff=128,
    e_layers=2,
    num_scales=3,
    top_k=3,
    dropout=0.1,
    down_sampling_window=2,
    down_sampling_layers=2,
    channel_independence=True,
    num_kernels=6,
)

epochs = 20
lr = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

exp_name = f"timemixer_pp_greenhouse_{team}_nb"
run_dir = create_run_dir(exp_name, base_dir=Path('./results'))
print(f"Experiment Dir: {run_dir}")

In [ ]:
prepared = prepare_greenhouse_datasets(
    dataset_root=dataset_root,
    team=team,
    seq_len=seq_len,
    pred_len=pred_len,
    stride=stride,
    missing_rate_threshold=0.7,
    drop_constant=True,
    protect_target_cols=True,
)
train_loader, val_loader, test_loader = make_loaders(prepared, batch_size=batch_size)
cfg.enc_in = len(prepared.feature_cols)
cfg.c_out = len(prepared.target_cols)
print(f"In: {cfg.enc_in}, Out: {cfg.c_out}")

In [ ]:
model = TimeMixerPPForecaster(cfg)

In [ ]:
trainer = Trainer(
    model=model,
    cfg=TrainConfig(
        epochs=epochs,
        lr=lr,
        device=device,
        ckpt_path=run_dir / 'checkpoints' / 'best.pt',
        early_stop_patience=6,
        show_progress=True
    )
)
history = trainer.fit(train_loader, val_loader=val_loader)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Train')
plt.plot(history['val_loss'], label='Val')
plt.legend()
plt.show()

metrics = trainer.evaluate(test_loader)

model.eval()
preds = []
trues = []
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        preds.append(model(x).cpu().numpy())
        trues.append(y.numpy())
        
y_hat = np.concatenate(preds)
y_true = np.concatenate(trues)
scaler = prepared.target_scaler
y_hat_raw = scaler.inverse_transform(y_hat.reshape(-1, y_hat.shape[-1])).reshape(y_hat.shape)
y_true_raw = scaler.inverse_transform(y_true.reshape(-1, y_true.shape[-1])).reshape(y_true.shape)

metrics['mae_raw'] = float(np.mean(np.abs(y_hat_raw - y_true_raw)))
metrics['rmse_raw'] = float(np.sqrt(np.mean((y_hat_raw - y_true_raw)**2)))

print(metrics)
save_metrics_json(run_dir, metrics)